In [1]:
import os

import swat
import pandas as pd

import networkx as nx

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.palettes import Blues8

from python_demo.datasets.examples import clique_graph_from_links
from python_demo.visualization.bokeh_graphs import plot_cliques, set_node_colors

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
host = os.environ['CAS_HOST']
port = int(os.environ['CAS_PORT'])
print(f"{host}:{port}")

rdcgrd113.unx.sas.com:23404


In [4]:
dfLinkSetIn = clique_graph_from_links()

In [5]:
conn = swat.CAS(host, port)

In [6]:
_ = conn.loadactionset('network')
_ = conn.loadactionset('fedsql')

NOTE: Added action set 'network'.
NOTE: Added action set 'fedsql'.


In [7]:
_ = conn.upload(dfLinkSetIn, casout=dict(name='LinkSetIn', caslib='CASUSER(daherr)'))

NOTE: Cloud Analytic Services made the uploaded file available as table LINKSETIN in caslib CASUSER(daherr).
NOTE: The table LINKSETIN has been created in caslib CASUSER(daherr) from binary data uploaded to Cloud Analytic Services.


In [8]:
conn.network.clique(links=dict(name='LinkSetIn', 
                               caslib='CASUSER(daherr)'),
                    outnodes=dict(name='nodeSetOut', 
                                  caslib='CASUSER(daherr)'), 
                    out=dict(name='cliques', 
                             caslib='CASUSER(daherr)'), 
                    maxcliques='ALL')

NOTE: The number of nodes in the input graph is 10.
NOTE: The number of links in the input graph is 19.
NOTE: Processing clique enumeration using 32 threads across 1 machines.
NOTE: The algorithm found 4 maximal cliques.
NOTE: Processing clique enumeration used 0.00 (cpu: 0.01) seconds.


[OutputCasTables]

             casLib        Name Label  Rows  Columns                                          casTable
 0  CASUSER(daherr)  nodeSetOut          10        1  CASTable('nodeSetOut', caslib='CASUSER(daherr)')
 1  CASUSER(daherr)     cliques          14        2     CASTable('cliques', caslib='CASUSER(daherr)')

[ProblemSummary]

 Problem Summary
 
             Name1           Label1       cValue1  nValue1
 0        numNodes  Number of Nodes            10     10.0
 1        numLinks  Number of Links            19     19.0
 2  graphDirection  Graph Direction    Undirected      NaN

[SolutionSummary]

 Solution Summary
 
          Name1             Label1       cValue1   nValue1
 0  problemType       Problem Type        Clique       NaN
 1       status    Solution Status            OK       NaN
 2   numCliques  Number of Cliques             4  4.000000
 3      cpuTime           CPU Time          0.01  0.010000
 4     realTime          Real Time          0.00  0.003351

[numCliques]

 4

[solutionStatus]

 'OK'

[solutionStatusCode]

 0

+ Elapsed: 0.00875s, user: 0.006s, sys: 0.014s, mem: 47.7mb

In [9]:
conn.fedsql.execdirect(query="create table cliqueSizes as select clique, count(*) from cliques group by clique")

NOTE: Table CLIQUESIZES was created in caslib CASUSER(daherr) with 4 rows returned.


+ Elapsed: 0.0569s, user: 0.118s, sys: 0.105s, mem: 117mb

In [10]:
conn.fetch(table=dict(name='cliqueSizes', caslib='CASUSER(daherr)'))

,clique,COUNT
0,3.0,2
1,2.0,3
2,1.0,4
3,0.0,5


We have the information we want now, so let's get the tables from CAS to our local notebook, and prep everything for plotting.

In [22]:
clique_count = list(conn.CASTable('cliqueSizes').to_dict()['clique'].keys())
print(clique_count)

[0, 1, 2, 3]


In [11]:
clique_list = conn.CASTable('cliques').to_dict(orient='records')

In [12]:
G = nx.from_pandas_edgelist(dfLinkSetIn, 'from', 'to')

In [13]:
for i in clique_list:
    node = int(i['node'])
    if 'clique' not in G.nodes[node]:
        G.nodes[node]['clique'] = [int(i['clique'])]
    else:
        G.nodes[node]['clique'].append(int(i['clique']))

In [14]:
#Choose a title!
title = 'Example Cliques'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("node", "@index"), ("Clique", "@clique")]

plot = plot_cliques(G, title, HOVER_TOOLTIPS)

show(plot)


In [23]:
# let's highlight the various cliques in this network.
palette = Blues8
attr_name = 'clique'
attr_for_highlight = 'highlight'
plot_list = []

for i in clique_count:
    title = f"Clique {i}"
    set_node_colors(G, i, attr_name, palette)
    plot_list.append(plot_cliques(G, title, HOVER_TOOLTIPS, attr_for_highlight))

columns = int(len(plot_list) / 2)
grid = gridplot(plot_list, ncols=columns)
show(grid)

In [ ]:
conn.droptable(name='LinkSetIn', caslib='CASUSER(daherr)', quiet=True)
conn.droptable(name='nodeSetOut', caslib='CASUSER(daherr)', quiet=True)
conn.droptable(name='cliques', caslib='CASUSER(daherr)', quiet=True)

In [ ]:
conn.close()